In [1]:
import torch
from torch import nn
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import seaborn as sns
import random
import requests
import pandas as pd
from pathlib import Path
import numpy as np
import os
import zipfile
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, balanced_accuracy_score, classification_report
from tqdm.auto import tqdm
from collections import Counter

In [2]:
# set device by checking if gpu is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [3]:
# set seed for reproducibilty
SEED = 50

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# # mount google drive
# from google.colab import drive
# drive.mount('/content/drive')

# # root directory where the food-11 zip file is located
# root_dir = 'drive/MyDrive/'

**DATA PREPROCESSING AND VISUALIZATION**

In [5]:
# extract the food-11 zip file
with zipfile.ZipFile('google_stock_price.zip', 'r') as zip_ref:
    zip_ref.extractall('google_stock_price/')

In [6]:
df = pd.read_csv('google_stock_price/Google_Stock_Price_Train.csv')
df.head(10)

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"
5,1/10/2012,313.70,315.72,307.30,621.43,"8,824,000"
6,1/11/2012,310.59,313.52,309.40,624.25,"4,817,800"
7,1/12/2012,314.43,315.26,312.08,627.92,"3,764,400"
8,1/13/2012,311.96,312.30,309.37,623.28,"4,631,800"
9,1/17/2012,314.81,314.81,311.67,626.86,"3,832,800"


In [7]:
class StockDataset(Dataset):
    def __init__(self, data, N):
        data['Volume'] = data['Volume'].str.replace(',', '')

        self.data = data[['Open', 'High', 'Low', 'Close', 'Volume']].apply(pd.to_numeric, errors='coerce')
        self.data.fillna(0, inplace=True)

        self.N = N

    def __len__(self):
        return len(self.data) - self.N

    def __getitem__(self, idx):
        # Get the sequence of features for the last M days
        x = self.data.iloc[idx:idx + self.N, :].values  # Exclude the last column 'Close'
        # Get the closing price of the next day as the target
        y = self.data.iloc[idx + self.N]['Close']

        # Convert to PyTorch tensors
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        return x, y

In [8]:
# Define your parameters
N = 30  # Number of days to look back
batch_size = 32  # Batch size for DataLoader

# Instantiate the dataset
train_data = StockDataset(df, N)

In [9]:
# Create the DataLoader
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [10]:
x,y = next(iter(train_dataloader))

print(x.shape,y.shape)

torch.Size([32, 30, 5]) torch.Size([32])


**TRAINING**

In [11]:
# function to train the model
def train(model, dataloader, criterion, optimizer):
    model.train()    # Set the model to training mode

    epoch_loss = 0  # Accumulate loss for each epoch

    for x, y in dataloader:
        optimizer.zero_grad()  # Zero the gradients

        # Forward pass
        preds = model(x)
        preds = preds.squeeze()  # Ensure predictions have the same shape as y_batch

        # Compute the loss
        loss = criterion(preds, y)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        # Accumulate loss for reporting
        epoch_loss += loss.item()

    # Print the average loss for each epoch
    avg_loss = epoch_loss / len(dataloader)

    return avg_loss

**BASELINE**

In [12]:
class Baseline(nn.Module):
    def __init__(self):

        super().__init__()

        self.rnn = nn.RNN(5,16,batch_first=True)

        self.fc = nn.Linear(16, 1)

    def forward(self, x):

        # embedded = [sent len, batch size, emb dim]

        output, hidden = self.rnn(x)

        hidden = hidden.squeeze()

        out = self.fc(hidden)

        return out

In [13]:
model = Baseline()

In [14]:
optimizer = torch.optim.Adam(model.parameters())

criterion = nn.MSELoss()

In [15]:
epochs = 50

In [16]:
for epoch in range(epochs):

    # Print the average loss for each epoch
    avg_loss = train(model, train_dataloader, criterion, optimizer)

    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

Epoch 1, Loss: 429967.6042
Epoch 2, Loss: 427456.6378
Epoch 3, Loss: 426077.4071
Epoch 4, Loss: 426220.6843
Epoch 5, Loss: 426368.9239
Epoch 6, Loss: 423470.0232
Epoch 7, Loss: 423969.4880
Epoch 8, Loss: 424377.6346
Epoch 9, Loss: 422053.1154
Epoch 10, Loss: 419398.0529
Epoch 11, Loss: 419909.0489
Epoch 12, Loss: 419548.4952
Epoch 13, Loss: 418758.0545
Epoch 14, Loss: 418419.8926
Epoch 15, Loss: 416879.4079
Epoch 16, Loss: 415961.3542
Epoch 17, Loss: 414401.4431
Epoch 18, Loss: 414987.1755
Epoch 19, Loss: 414599.3237
Epoch 20, Loss: 412914.9447
Epoch 21, Loss: 412933.8814
Epoch 22, Loss: 413266.5793
Epoch 23, Loss: 408850.2829
Epoch 24, Loss: 409559.1595
Epoch 25, Loss: 411028.1410
Epoch 26, Loss: 407844.9960
Epoch 27, Loss: 411126.6979
Epoch 28, Loss: 406195.9279
Epoch 29, Loss: 405605.5144
Epoch 30, Loss: 406545.2372
Epoch 31, Loss: 403587.0609
Epoch 32, Loss: 405275.7500
Epoch 33, Loss: 404416.5921
Epoch 34, Loss: 402968.2604
Epoch 35, Loss: 402419.3045
Epoch 36, Loss: 399891.3646
E

**LSTM**

In [17]:
class LSTM(nn.Module):
    def __init__(self):

        super().__init__()

        self.rnn = nn.LSTM(5,
                           64,
                           batch_first=True,
                           num_layers=2)

        self.fc = nn.Linear(128, 1)

    def forward(self, x):

        # embedded = [sent len, batch size, emb dim]

        output, (hidden,cell) = self.rnn(x)

        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)

        out = self.fc(hidden)

        return out

In [18]:
model = LSTM()

In [19]:
optimizer = torch.optim.Adam(model.parameters())

criterion = nn.MSELoss()

In [20]:
epochs = 50

In [21]:
for epoch in range(epochs):

    # Print the average loss for each epoch
    avg_loss = train(model, train_dataloader, criterion, optimizer)

    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

Epoch 1, Loss: 427521.3133
Epoch 2, Loss: 420490.7420
Epoch 3, Loss: 415799.1627
Epoch 4, Loss: 411603.3213
Epoch 5, Loss: 406322.4880
Epoch 6, Loss: 405504.2772
Epoch 7, Loss: 400637.6194
Epoch 8, Loss: 396870.9303
Epoch 9, Loss: 392493.7580
Epoch 10, Loss: 388542.1971
Epoch 11, Loss: 385229.8694
Epoch 12, Loss: 380434.2933
Epoch 13, Loss: 379734.3934
Epoch 14, Loss: 374027.2780
Epoch 15, Loss: 371284.3734
Epoch 16, Loss: 367389.6771
Epoch 17, Loss: 365464.8694
Epoch 18, Loss: 362091.3774
Epoch 19, Loss: 357450.0865
Epoch 20, Loss: 355628.7179
Epoch 21, Loss: 352056.9054
Epoch 22, Loss: 347937.4704
Epoch 23, Loss: 346085.8990
Epoch 24, Loss: 342219.2412
Epoch 25, Loss: 339886.6438
Epoch 26, Loss: 336678.3085
Epoch 27, Loss: 333912.9655
Epoch 28, Loss: 330201.2043
Epoch 29, Loss: 327732.9952
Epoch 30, Loss: 324440.4479
Epoch 31, Loss: 320846.4062
Epoch 32, Loss: 318030.0176
Epoch 33, Loss: 314615.0369
Epoch 34, Loss: 312738.4720
Epoch 35, Loss: 308916.4411
Epoch 36, Loss: 306181.9704
E